# Neural Network Analysis

**Author**: Maleakhi Agung Wijaya  
**Email**: *maw219@cam.ac.uk*  
**Description**: This file contains code implementation of various neural networks, including 2D-CNNPred, 3D-CNNPred, shallow feedforward network, and RNN-based network.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from os.path import join
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, mean_absolute_error as mae
import os
from pathlib2 import Path
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers, models, backend as K, callbacks

In [2]:
%run Utilities.ipynb

## Load data and preprocessing

In this step, we will load all dataframes,fill missing values, and scale so that each column all features are on the same scale. Afterwards, we will generate sequential datasets using **full features, PCA features**, and only **technical indicator features**.

In [3]:
market_orders, n_markets, aggregated_datasets = load_aggregated_datasets([DATASET_DJI, 
                                                                          DATASET_NASDAQ, 
                                                                          DATASET_NYSE,
                                                                          DATASET_RUSSELL, 
                                                                          DATASET_SP])

# Load datasets
## DJI
dji_df = aggregated_datasets["DJI"]

## NASDAQ
nasdaq_df = aggregated_datasets["NASDAQ"]

## NYSE
nyse_df = aggregated_datasets["NYA"]

## Russell
russell_df = aggregated_datasets["RUT"]

## SP
sp_df = aggregated_datasets["S&P"]

In [4]:
## DO NOT RUN BELOW FOR 3D CNN PRED
# Fill missing values, do some scaling (run prev cell first)
list_df = []

for df in [dji_df, nasdaq_df, nyse_df, russell_df, sp_df]:
    columns = df.columns
    df.fillna(0, inplace=True) # fill na with 0
    y = df["MOVEMENT"].copy()
    X = df.drop(columns=["MOVEMENT"]).copy()
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X))
    X["MOVEMENT"] = np.array(y)
    X.columns = columns
    list_df.append(X)
    
### Clean dataframe (full features)
dji_df_full = list_df[0]
nasdaq_df_full = list_df[1]
nyse_df_full = list_df[2]
russell_df_full = list_df[3]
sp_df_full = list_df[4]

In [5]:
# PCA dataframe (30 features - explained 90% variance) - pca.explained_variance_ratio_.cumsum()
list_df_pca = []

for df in [dji_df_full, nasdaq_df_full, nyse_df_full, russell_df_full, sp_df_full]:
    pca = PCA(n_components=30)
    y = df["MOVEMENT"].copy()
    X = df.drop(columns=["MOVEMENT"]).copy()
    reduced_X = pd.DataFrame(pca.fit_transform(X))
    reduced_X["MOVEMENT"] = y
    list_df_pca.append(reduced_X)

### Clean dataframe (pca features)
dji_df_pca = list_df_pca[0]
nasdaq_df_pca = list_df_pca[1]
nyse_df_pca = list_df_pca[2]
russell_df_pca = list_df_pca[3]
sp_df_pca = list_df_pca[4]

In [6]:
# Technical indicator dataframe
ti_columns = ["Volume", "mom", "mom1", "mom2", "mom3", 
              "ROC_5", "ROC_10", "ROC_15", "ROC_20",
              "EMA_10", "EMA_20", "EMA_50", "EMA_200"]
list_df_ti = []

for df in [dji_df_full, nasdaq_df_full, nyse_df_full, russell_df_full, sp_df_full]:
    y = df["MOVEMENT"].copy()
    X = df[ti_columns].copy()
    X["MOVEMENT"] = np.array(y)
    list_df_ti.append(X)

### Clean dataframe (ti features)
dji_df_ti = list_df_ti[0]
nasdaq_df_ti = list_df_ti[1]
nyse_df_ti = list_df_ti[2]
russell_df_ti = list_df_ti[3]
sp_df_ti = list_df_ti[4]

In [7]:
# Build sequential dataset
sequence_length = 60

### Sequential dataset (full features)
dji_X_seq, dji_y_seq = generate_sequential_data(dji_df_full, sequence_length)
nasdaq_X_seq, nasdaq_y_seq = generate_sequential_data(nasdaq_df_full, sequence_length)
nyse_X_seq, nyse_y_seq = generate_sequential_data(nyse_df_full, sequence_length)
russell_X_seq, russell_y_seq = generate_sequential_data(russell_df_full, sequence_length)
sp_X_seq, sp_y_seq = generate_sequential_data(sp_df_full, sequence_length)

In [8]:
### Sequential dataset (PCA features)
dji_X_pca_seq, dji_y_pca_seq = generate_sequential_data(dji_df_pca, sequence_length)
nasdaq_X_pca_seq, nasdaq_y_pca_seq = generate_sequential_data(nasdaq_df_pca, sequence_length)
nyse_X_pca_seq, nyse_y_pca_seq = generate_sequential_data(nyse_df_pca, sequence_length)
russell_X_pca_seq, russell_y_pca_seq = generate_sequential_data(russell_df_pca, sequence_length)
sp_X_pca_seq, sp_y_pca_seq = generate_sequential_data(sp_df_pca, sequence_length)

In [9]:
### Sequential dataset (TI features)
dji_X_ti_seq, dji_y_ti_seq = generate_sequential_data(dji_df_ti, sequence_length)
nasdaq_X_ti_seq, nasdaq_y_ti_seq = generate_sequential_data(nasdaq_df_ti, sequence_length)
nyse_X_ti_seq, nyse_y_ti_seq = generate_sequential_data(nyse_df_ti, sequence_length)
russell_X_ti_seq, russell_y_ti_seq = generate_sequential_data(russell_df_ti, sequence_length)
sp_X_ti_seq, sp_y_ti_seq = generate_sequential_data(sp_df_ti, sequence_length)

## 2D CNNpred

In this section, we will implement a 2D CNNpred from the paper.

### Load data

In [10]:
# Sequential flatten (full features)
dji_X_seq_flatten = sequential_reshape(dji_X_seq, (len(dji_X_seq), sequence_length, -1, 1))
nasdaq_X_seq_flatten = sequential_reshape(nasdaq_X_seq, (len(dji_X_seq), sequence_length, -1, 1))
nyse_X_seq_flatten = sequential_reshape(nyse_X_seq, (len(dji_X_seq), sequence_length, -1, 1))
russell_X_seq_flatten = sequential_reshape(russell_X_seq, (len(dji_X_seq), sequence_length, -1, 1))
sp_X_seq_flatten = sequential_reshape(sp_X_seq, (len(dji_X_seq), sequence_length, -1, 1))

In [11]:
# Sequential flatten (pca)
dji_X_pca_seq_flatten = sequential_reshape(dji_X_pca_seq, (len(dji_X_seq), sequence_length, -1, 1))
nasdaq_X_pca_seq_flatten = sequential_reshape(nasdaq_X_pca_seq, (len(dji_X_seq), sequence_length, -1, 1))
nyse_X_pca_seq_flatten = sequential_reshape(nyse_X_pca_seq, (len(dji_X_seq), sequence_length, -1, 1))
russell_X_pca_seq_flatten = sequential_reshape(russell_X_pca_seq, (len(dji_X_seq), sequence_length, -1, 1))
sp_X_pca_seq_flatten = sequential_reshape(sp_X_pca_seq, (len(dji_X_seq), sequence_length, -1, 1))

In [12]:
# Sequential flatten (technical indicator)
dji_X_ti_seq_flatten = sequential_reshape(dji_X_ti_seq, (len(dji_X_seq), sequence_length, -1, 1))
nasdaq_X_ti_seq_flatten = sequential_reshape(nasdaq_X_ti_seq, (len(dji_X_seq), sequence_length, -1, 1))
nyse_X_ti_seq_flatten = sequential_reshape(nyse_X_ti_seq, (len(dji_X_seq), sequence_length, -1, 1))
russell_X_ti_seq_flatten = sequential_reshape(russell_X_ti_seq, (len(dji_X_seq), sequence_length, -1, 1))
sp_X_ti_seq_flatten = sequential_reshape(sp_X_ti_seq, (len(dji_X_seq), sequence_length, -1, 1))

### Split into training, validation, and testing (80/10/10)

In [13]:
## Full features
dji_X_train_full, dji_X_test_full, dji_y_train_full, dji_y_test_full = train_test_split(dji_X_seq_flatten,
                                                                                        dji_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1,
                                                                                        shuffle=False)
dji_X_train_full, dji_X_valid_full, dji_y_train_full, dji_y_valid_full = train_test_split(dji_X_train_full,
                                                                                        dji_y_train_full,
                                                                                        stratify=None,
                                                                                        test_size=0.1,
                                                                                        shuffle=False)
nasdaq_X_train_full, nasdaq_X_test_full, nasdaq_y_train_full, nasdaq_y_test_full = train_test_split(nasdaq_X_seq_flatten,
                                                                                        nasdaq_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1,
                                                                                        shuffle=False)
nasdaq_X_train_full, nasdaq_X_valid_full, nasdaq_y_train_full, nasdaq_y_valid_full = train_test_split(nasdaq_X_train_full,
                                                                                        nasdaq_y_train_full,
                                                                                        stratify=None,
                                                                                        test_size=0.1,
                                                                                        shuffle=False)
nyse_X_train_full, nyse_X_test_full, nyse_y_train_full, nyse_y_test_full = train_test_split(nyse_X_seq_flatten,
                                                                                        nyse_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nyse_X_train_full, nyse_X_valid_full, nyse_y_train_full, nyse_y_valid_full = train_test_split(nyse_X_train_full,
                                                                                        nyse_y_train_full,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)

russell_X_train_full, russell_X_test_full, russell_y_train_full, russell_y_test_full = train_test_split(russell_X_seq_flatten,
                                                                                        russell_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
russell_X_train_full, russell_X_valid_full, russell_y_train_full, russell_y_valid_full = train_test_split(russell_X_train_full,
                                                                                        russell_y_train_full,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
sp_X_train_full, sp_X_test_full, sp_y_train_full, sp_y_test_full = train_test_split(sp_X_seq_flatten,
                                                                                        sp_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
sp_X_train_full, sp_X_valid_full, sp_y_train_full, sp_y_valid_full = train_test_split(sp_X_train_full,
                                                                                        sp_y_train_full,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)

In [14]:
dji_X_train_full.shape

(1395, 60, 82, 1)

In [15]:
dji_X_valid_full.shape

(156, 60, 82, 1)

In [16]:
dji_X_test_full.shape

(173, 60, 82, 1)

In [14]:
## pca features
dji_X_train_pca, dji_X_test_pca, dji_y_train_pca, dji_y_test_pca = train_test_split(dji_X_pca_seq_flatten,
                                                                                        dji_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
dji_X_train_pca, dji_X_valid_pca, dji_y_train_pca, dji_y_valid_pca = train_test_split(dji_X_train_pca,
                                                                                        dji_y_train_pca,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nasdaq_X_train_pca, nasdaq_X_test_pca, nasdaq_y_train_pca, nasdaq_y_test_pca = train_test_split(nasdaq_X_pca_seq_flatten,
                                                                                        nasdaq_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nasdaq_X_train_pca, nasdaq_X_valid_pca, nasdaq_y_train_pca, nasdaq_y_valid_pca = train_test_split(nasdaq_X_train_pca,
                                                                                        nasdaq_y_train_pca,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nyse_X_train_pca, nyse_X_test_pca, nyse_y_train_pca, nyse_y_test_pca = train_test_split(nyse_X_pca_seq_flatten,
                                                                                        nyse_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nyse_X_train_pca, nyse_X_valid_pca, nyse_y_train_pca, nyse_y_valid_pca = train_test_split(nyse_X_train_pca,
                                                                                        nyse_y_train_pca,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
russell_X_train_pca, russell_X_test_pca, russell_y_train_pca, russell_y_test_pca = train_test_split(russell_X_pca_seq_flatten,
                                                                                        russell_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
russell_X_train_pca, russell_X_valid_pca, russell_y_train_pca, russell_y_valid_pca = train_test_split(russell_X_train_pca,
                                                                                        russell_y_train_pca,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
sp_X_train_pca, sp_X_test_pca, sp_y_train_pca, sp_y_test_pca = train_test_split(sp_X_pca_seq_flatten,
                                                                                        sp_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
sp_X_train_pca, sp_X_valid_pca, sp_y_train_pca, sp_y_valid_pca = train_test_split(sp_X_train_pca,
                                                                                        sp_y_train_pca,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)

In [15]:
## ti features
dji_X_train_ti, dji_X_test_ti, dji_y_train_ti, dji_y_test_ti = train_test_split(dji_X_ti_seq_flatten,
                                                                                        dji_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
dji_X_train_ti, dji_X_valid_ti, dji_y_train_ti, dji_y_valid_ti = train_test_split(dji_X_train_ti,
                                                                                        dji_y_train_ti,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nasdaq_X_train_ti, nasdaq_X_test_ti, nasdaq_y_train_ti, nasdaq_y_test_ti = train_test_split(nasdaq_X_ti_seq_flatten,
                                                                                        nasdaq_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nasdaq_X_train_ti, nasdaq_X_valid_ti, nasdaq_y_train_ti, nasdaq_y_valid_ti = train_test_split(nasdaq_X_train_ti,
                                                                                        nasdaq_y_train_ti,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nyse_X_train_ti, nyse_X_test_ti, nyse_y_train_ti, nyse_y_test_ti = train_test_split(nyse_X_ti_seq_flatten,
                                                                                        nyse_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
nyse_X_train_ti, nyse_X_valid_ti, nyse_y_train_ti, nyse_y_valid_ti = train_test_split(nyse_X_train_ti,
                                                                                        nyse_y_train_ti,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
russell_X_train_ti, russell_X_test_ti, russell_y_train_ti, russell_y_test_ti = train_test_split(russell_X_ti_seq_flatten,
                                                                                        russell_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
russell_X_train_ti, russell_X_valid_ti, russell_y_train_ti, russell_y_valid_ti = train_test_split(russell_X_train_ti,
                                                                                        russell_y_train_ti,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
sp_X_train_ti, sp_X_test_ti, sp_y_train_ti, sp_y_test_ti = train_test_split(sp_X_ti_seq_flatten,
                                                                                        sp_y_seq,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)
sp_X_train_ti, sp_X_valid_ti, sp_y_train_ti, sp_y_valid_ti = train_test_split(sp_X_train_ti,
                                                                                        sp_y_train_ti,
                                                                                        stratify=None,
                                                                                        test_size=0.1, shuffle=False)

### Combine datasets

In [16]:
# full
# Train
X_train_full = np.concatenate(
    (dji_X_train_full,
     nasdaq_X_train_full,
     nyse_X_train_full,
     russell_X_train_full,
     sp_X_train_full)
)
y_train_full = np.concatenate(
    (np.array(dji_y_train_full),
     np.array(nasdaq_y_train_full),
     np.array(nyse_y_train_full),
     np.array(russell_y_train_full),
     np.array(sp_y_train_full)
    )
)

# Valid
X_valid_full = np.concatenate(
    (dji_X_valid_full,
     nasdaq_X_valid_full,
     nyse_X_valid_full,
     russell_X_valid_full,
     sp_X_valid_full)
)
y_valid_full = np.concatenate(
    (np.array(dji_y_valid_full),
     np.array(nasdaq_y_valid_full),
     np.array(nyse_y_valid_full),
     np.array(russell_y_valid_full),
     np.array(sp_y_valid_full)
    )
)

In [17]:
# pca
# Train
X_train_pca = np.concatenate(
    (dji_X_train_pca,
     nasdaq_X_train_pca,
     nyse_X_train_pca,
     russell_X_train_pca,
     sp_X_train_pca)
)
y_train_pca = np.concatenate(
    (np.array(dji_y_train_pca),
     np.array(nasdaq_y_train_pca),
     np.array(nyse_y_train_pca),
     np.array(russell_y_train_pca),
     np.array(sp_y_train_pca)
    )
)

# Valid
X_valid_pca = np.concatenate(
    (dji_X_valid_pca,
     nasdaq_X_valid_pca,
     nyse_X_valid_pca,
     russell_X_valid_pca,
     sp_X_valid_pca)
)
y_valid_pca = np.concatenate(
    (np.array(dji_y_valid_pca),
     np.array(nasdaq_y_valid_pca),
     np.array(nyse_y_valid_pca),
     np.array(russell_y_valid_pca),
     np.array(sp_y_valid_pca)
    )
)

In [18]:
# ti
# Train
X_train_ti = np.concatenate(
    (dji_X_train_ti,
     nasdaq_X_train_ti,
     nyse_X_train_ti,
     russell_X_train_ti,
     sp_X_train_ti)
)
y_train_ti = np.concatenate(
    (np.array(dji_y_train_ti),
     np.array(nasdaq_y_train_ti),
     np.array(nyse_y_train_ti),
     np.array(russell_y_train_ti),
     np.array(sp_y_train_ti)
    )
)

# Valid
X_valid_ti = np.concatenate(
    (dji_X_valid_ti,
     nasdaq_X_valid_ti,
     nyse_X_valid_ti,
     russell_X_valid_ti,
     sp_X_valid_ti)
)
y_valid_ti = np.concatenate(
    (np.array(dji_y_valid_ti),
     np.array(nasdaq_y_valid_ti),
     np.array(nyse_y_valid_ti),
     np.array(russell_y_valid_ti),
     np.array(sp_y_valid_ti)
    )
)

### Model training and evaluation

#### Full features

In [37]:
model_full = cnnpred_2d(60, 82, [8, 8, 8])
epochs = 200
batch_size=128

## Training
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model_full.compile(optimizer="Adam", loss="binary_crossentropy", 
                   metrics=["acc", f1])
model_full.fit(X_train_full, y_train_full, epochs=epochs,
              batch_size=batch_size, callbacks=[early_stopping],
              validation_data=(X_valid_full, y_valid_full))

Epoch 1/200
55/55 [==============================] - 1s 11ms/step - loss: 0.6883 - acc: 0.5385 - f1: 0.4819 - val_loss: 0.6927 - val_acc: 0.5308 - val_f1: 0.3660
Epoch 2/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6803 - acc: 0.5663 - f1: 0.5236 - val_loss: 0.7061 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 3/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6729 - acc: 0.5882 - f1: 0.5362 - val_loss: 0.6946 - val_acc: 0.5321 - val_f1: 0.3951
Epoch 4/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6619 - acc: 0.6080 - f1: 0.5777 - val_loss: 0.6957 - val_acc: 0.5321 - val_f1: 0.4547
Epoch 5/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6514 - acc: 0.6288 - f1: 0.6089 - val_loss: 0.6971 - val_acc: 0.5256 - val_f1: 0.4590
Epoch 6/200
55/55 [==============================] - 0s 7ms/step - loss: 0.6362 - acc: 0.6462 - f1: 0.6333 - val_loss: 0.7012 - val_acc: 0.5359 - val_f1: 0.4856
Epoch 7/200
55/55 [==============

In [39]:
## DJI
result_dji_full = model_full.predict(dji_X_test_full)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"DJI Accuracy: {accuracy_score(result_dji_full, dji_y_test_full)}")
print(f"DJI F1: {f1_score(result_dji_full, dji_y_test_full, average='macro')}")

DJI Accuracy: 0.5317919075144508
DJI F1: 0.4891173575412884


In [40]:
## NASDAQ
result_nasdaq_full = model_full.predict(nasdaq_X_test_full)
result_nasdaq_full = (result_nasdaq_full > 0.5).astype(int)
print(f"NASDAQ Accuracy: {accuracy_score(result_nasdaq_full, nasdaq_y_test_full)}")
print(f"NASDAQ F1: {f1_score(result_nasdaq_full, nasdaq_y_test_full, average='macro')}")

NASDAQ Accuracy: 0.5780346820809249
NASDAQ F1: 0.5246000376435158


In [41]:
## nyse
result_nyse_full = model_full.predict(nyse_X_test_full)
result_nyse_full = (result_nyse_full > 0.5).astype(int)
print(f"nyse Accuracy: {accuracy_score(result_nyse_full, nyse_y_test_full)}")
print(f"nyse F1: {f1_score(result_nyse_full, nyse_y_test_full, average='macro')}")

nyse Accuracy: 0.5202312138728323
nyse F1: 0.47650297130773994


In [42]:
## russell
result_russell_full = model_full.predict(russell_X_test_full)
result_russell_full = (result_russell_full > 0.5).astype(int)
print(f"russell Accuracy: {accuracy_score(result_russell_full, russell_y_test_full)}")
print(f"russell F1: {f1_score(result_russell_full, russell_y_test_full, average='macro')}")

russell Accuracy: 0.5317919075144508
russell F1: 0.4852892561983471


In [43]:
## sp
result_sp_full = model_full.predict(sp_X_test_full)
result_sp_full = (result_sp_full > 0.5).astype(int)
print(f"S&P 500 Accuracy: {accuracy_score(result_sp_full, sp_y_test_full)}")
print(f"S&P 500 F1: {f1_score(result_sp_full, sp_y_test_full, average='macro')}")

S&P 500 Accuracy: 0.5375722543352601
S&P 500 F1: 0.4896755162241888


#### PCA features

In [50]:
model_pca = cnnpred_2d(60, 30, [8, 8, 8])
epochs = 200
batch_size=128

## Training
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model_pca.compile(optimizer="Adam", loss="mae", 
                   metrics=["acc", f1])
model_pca.fit(X_train_pca, y_train_pca, epochs=epochs,
              batch_size=batch_size, callbacks=[early_stopping],
              validation_data=(X_valid_pca, y_valid_pca))

Epoch 1/200
55/55 [==============================] - 1s 9ms/step - loss: 0.4799 - acc: 0.5372 - f1: 0.3609 - val_loss: 0.4758 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 2/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4648 - acc: 0.5379 - f1: 0.3491 - val_loss: 0.4671 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 3/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4628 - acc: 0.5379 - f1: 0.3496 - val_loss: 0.4654 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 4/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4623 - acc: 0.5379 - f1: 0.3493 - val_loss: 0.4647 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 5/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4622 - acc: 0.5379 - f1: 0.3489 - val_loss: 0.4644 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 6/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4621 - acc: 0.5379 - f1: 0.3495 - val_loss: 0.4642 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 7/200
55/55 [===============

Epoch 102/200
55/55 [==============================] - 0s 6ms/step - loss: 0.2815 - acc: 0.7267 - f1: 0.7138 - val_loss: 0.5570 - val_acc: 0.4385 - val_f1: 0.4627
Epoch 103/200
55/55 [==============================] - 0s 6ms/step - loss: 0.2803 - acc: 0.7302 - f1: 0.7184 - val_loss: 0.5568 - val_acc: 0.4423 - val_f1: 0.4656
Epoch 104/200
55/55 [==============================] - 0s 6ms/step - loss: 0.2780 - acc: 0.7299 - f1: 0.7179 - val_loss: 0.5576 - val_acc: 0.4321 - val_f1: 0.4414
Epoch 105/200
55/55 [==============================] - 0s 6ms/step - loss: 0.2832 - acc: 0.7239 - f1: 0.7093 - val_loss: 0.5543 - val_acc: 0.4397 - val_f1: 0.4638
Epoch 106/200
55/55 [==============================] - 0s 7ms/step - loss: 0.2776 - acc: 0.7309 - f1: 0.7197 - val_loss: 0.5537 - val_acc: 0.4487 - val_f1: 0.4752
Epoch 107/200
55/55 [==============================] - 0s 6ms/step - loss: 0.2777 - acc: 0.7306 - f1: 0.7202 - val_loss: 0.5539 - val_acc: 0.4487 - val_f1: 0.4659
Epoch 108/200
55/55 [=

In [51]:
## DJI
result_dji_pca = model_pca.predict(dji_X_test_pca)
result_dji_pca = (result_dji_pca > 0.5).astype(int)
print(f"DJI Accuracy: {accuracy_score(result_dji_pca, dji_y_test_pca)}")
print(f"DJI F1: {f1_score(result_dji_pca, dji_y_test_pca, average='macro')}")

DJI Accuracy: 0.48554913294797686
DJI F1: 0.4770928850399049


In [52]:
## NASDAQ
result_nasdaq_pca = model_pca.predict(nasdaq_X_test_pca)
result_nasdaq_pca = (result_nasdaq_pca > 0.5).astype(int)
print(f"NASDAQ Accuracy: {accuracy_score(result_nasdaq_pca, nasdaq_y_test_pca)}")
print(f"NASDAQ F1: {f1_score(result_nasdaq_pca, nasdaq_y_test_pca, average='macro')}")

NASDAQ Accuracy: 0.5433526011560693
NASDAQ F1: 0.5175615094073212


In [53]:
## nyse
result_nyse_pca = model_pca.predict(nyse_X_test_pca)
result_nyse_pca = (result_nyse_pca > 0.5).astype(int)
print(f"nyse Accuracy: {accuracy_score(result_nyse_pca, nyse_y_test_pca)}")
print(f"nyse F1: {f1_score(result_nyse_pca, nyse_y_test_pca, average='macro')}")

nyse Accuracy: 0.5086705202312138
nyse F1: 0.49902906006200387


In [54]:
## russell
result_russell_pca = model_pca.predict(russell_X_test_pca)
result_russell_pca = (result_russell_pca > 0.5).astype(int)
print(f"russell Accuracy: {accuracy_score(result_russell_pca, russell_y_test_pca)}")
print(f"russell F1: {f1_score(result_russell_pca, russell_y_test_pca, average='macro')}")

russell Accuracy: 0.4797687861271676
russell F1: 0.46678082191780823


In [55]:
## sp
result_sp_pca = model_pca.predict(sp_X_test_pca)
result_sp_pca = (result_sp_pca > 0.5).astype(int)
print(f"S&P 500 Accuracy: {accuracy_score(result_sp_pca, sp_y_test_pca)}")
print(f"S&P 500 F1: {f1_score(result_sp_pca, sp_y_test_pca, average='macro')}")

S&P 500 Accuracy: 0.5260115606936416
S&P 500 F1: 0.5189229517091698


#### Technical indicator features

In [68]:
model_ti = cnnpred_2d(60, 13, [8, 8, 8])
epochs = 200
batch_size=128

## Training
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=50, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model_ti.compile(optimizer="Adam", loss="mae", 
                   metrics=["acc", f1])
model_ti.fit(X_train_ti, y_train_ti, epochs=epochs,
              batch_size=batch_size, callbacks=[early_stopping],
              validation_data=(X_valid_ti, y_valid_ti))

Epoch 1/200
55/55 [==============================] - 1s 9ms/step - loss: 0.4973 - acc: 0.5143 - f1: 0.4387 - val_loss: 0.4915 - val_acc: 0.5449 - val_f1: 0.3713
Epoch 2/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4751 - acc: 0.5385 - f1: 0.3550 - val_loss: 0.4730 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 3/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4641 - acc: 0.5379 - f1: 0.3493 - val_loss: 0.4657 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 4/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4621 - acc: 0.5379 - f1: 0.3492 - val_loss: 0.4656 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 5/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4622 - acc: 0.5379 - f1: 0.3490 - val_loss: 0.4655 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 6/200
55/55 [==============================] - 0s 6ms/step - loss: 0.4619 - acc: 0.5381 - f1: 0.3496 - val_loss: 0.4659 - val_acc: 0.5372 - val_f1: 0.3516
Epoch 7/200
55/55 [===============

In [69]:
## DJI
result_dji_ti = model_ti.predict(dji_X_test_ti)
result_dji_ti = (result_dji_ti > 0.5).astype(int)
print(f"DJI Accuracy: {accuracy_score(result_dji_ti, dji_y_test_ti)}")
print(f"DJI F1: {f1_score(result_dji_ti, dji_y_test_ti, average='macro')}")

DJI Accuracy: 0.47398843930635837
DJI F1: 0.4682317176152677


In [70]:
## NASDAQ
result_nasdaq_ti = model_ti.predict(nasdaq_X_test_ti)
result_nasdaq_ti = (result_nasdaq_ti > 0.5).astype(int)
print(f"NASDAQ Accuracy: {accuracy_score(result_nasdaq_ti, nasdaq_y_test_ti)}")
print(f"NASDAQ F1: {f1_score(result_nasdaq_ti, nasdaq_y_test_ti, average='macro')}")

NASDAQ Accuracy: 0.4682080924855491
NASDAQ F1: 0.45056614194973765


In [71]:
## nyse
result_nyse_ti = model_ti.predict(nyse_X_test_ti)
result_nyse_ti = (result_nyse_ti > 0.5).astype(int)
print(f"nyse Accuracy: {accuracy_score(result_nyse_ti, nyse_y_test_ti)}")
print(f"nyse F1: {f1_score(result_nyse_ti, nyse_y_test_ti, average='macro')}")

nyse Accuracy: 0.5028901734104047
nyse F1: 0.48639878486605914


In [72]:
## russell
result_russell_ti = model_ti.predict(russell_X_test_ti)
result_russell_ti = (result_russell_ti > 0.5).astype(int)
print(f"russell Accuracy: {accuracy_score(result_russell_ti, russell_y_test_ti)}")
print(f"russell F1: {f1_score(result_russell_ti, russell_y_test_ti, average='macro')}")

russell Accuracy: 0.4682080924855491
russell F1: 0.46417990842984114


In [73]:
## sp
result_sp_ti = model_ti.predict(sp_X_test_ti)
result_sp_ti = (result_sp_ti > 0.5).astype(int)
print(f"S&P 500 Accuracy: {accuracy_score(result_sp_ti, sp_y_test_ti)}")
print(f"S&P 500 F1: {f1_score(result_sp_ti, sp_y_test_ti, average='macro')}")

S&P 500 Accuracy: 0.48554913294797686
S&P 500 F1: 0.4770928850399049


## Feedforward Neural Networks

### Load data

In [105]:
# Sequential flatten (full features)
dji_X_seq_flatten = sequential_reshape(dji_X_seq, (len(dji_X_seq), -1))
nasdaq_X_seq_flatten = sequential_reshape(nasdaq_X_seq, (len(nasdaq_X_seq), -1))
nyse_X_seq_flatten = sequential_reshape(nyse_X_seq, (len(nyse_X_seq), -1))
russell_X_seq_flatten = sequential_reshape(russell_X_seq, (len(russell_X_seq), -1))
sp_X_seq_flatten = sequential_reshape(sp_X_seq, (len(sp_X_seq), -1))

In [106]:
# Sequential flatten (pca)
dji_X_pca_seq_flatten = sequential_reshape(dji_X_pca_seq, (len(dji_X_pca_seq), -1))
nasdaq_X_pca_seq_flatten = sequential_reshape(nasdaq_X_pca_seq, (len(nasdaq_X_pca_seq), -1))
nyse_X_pca_seq_flatten = sequential_reshape(nyse_X_pca_seq, (len(nyse_X_pca_seq), -1))
russell_X_pca_seq_flatten = sequential_reshape(russell_X_pca_seq, (len(russell_X_pca_seq), -1))
sp_X_pca_seq_flatten = sequential_reshape(sp_X_pca_seq, (len(sp_X_pca_seq), -1))

In [107]:
# Sequential flatten (technical indicator)
dji_X_ti_seq_flatten = sequential_reshape(dji_X_ti_seq, (len(dji_X_ti_seq), -1))
nasdaq_X_ti_seq_flatten = sequential_reshape(nasdaq_X_ti_seq, (len(nasdaq_X_ti_seq), -1))
nyse_X_ti_seq_flatten = sequential_reshape(nyse_X_ti_seq, (len(nyse_X_ti_seq), -1))
russell_X_ti_seq_flatten = sequential_reshape(russell_X_ti_seq, (len(russell_X_ti_seq), -1))
sp_X_ti_seq_flatten = sequential_reshape(sp_X_ti_seq, (len(sp_X_ti_seq), -1))

### Split into training/validation/test (80/10/10)

In [108]:
## Full features
dji_X_train_full, dji_X_test_full, dji_y_train_full, dji_y_test_full = train_test_split(dji_X_seq_flatten,
                                                                                        dji_y_seq,
                                                                                        stratify=dji_y_seq,
                                                                                        test_size=0.1,
                                                                                        shuffle=True)
dji_X_train_full, dji_X_valid_full, dji_y_train_full, dji_y_valid_full = train_test_split(dji_X_train_full,
                                                                                        dji_y_train_full,
                                                                                        stratify=dji_y_train_full,
                                                                                        test_size=0.1,
                                                                                        shuffle=True)
nasdaq_X_train_full, nasdaq_X_test_full, nasdaq_y_train_full, nasdaq_y_test_full = train_test_split(nasdaq_X_seq_flatten,
                                                                                        nasdaq_y_seq,
                                                                                        stratify=nasdaq_y_seq,
                                                                                        test_size=0.1,
                                                                                        shuffle=True)
nasdaq_X_train_full, nasdaq_X_valid_full, nasdaq_y_train_full, nasdaq_y_valid_full = train_test_split(nasdaq_X_train_full,
                                                                                        nasdaq_y_train_full,
                                                                                        stratify=nasdaq_y_train_full,
                                                                                        test_size=0.1,
                                                                                        shuffle=True)
nyse_X_train_full, nyse_X_test_full, nyse_y_train_full, nyse_y_test_full = train_test_split(nyse_X_seq_flatten,
                                                                                        nyse_y_seq,
                                                                                        stratify=nyse_y_seq,
                                                                                        test_size=0.1, shuffle=True)
nyse_X_train_full, nyse_X_valid_full, nyse_y_train_full, nyse_y_valid_full = train_test_split(nyse_X_train_full,
                                                                                        nyse_y_train_full,
                                                                                        stratify=nyse_y_train_full,
                                                                                        test_size=0.1, shuffle=True)

russell_X_train_full, russell_X_test_full, russell_y_train_full, russell_y_test_full = train_test_split(russell_X_seq_flatten,
                                                                                        russell_y_seq,
                                                                                        stratify=russell_y_seq,
                                                                                        test_size=0.1, shuffle=True)
russell_X_train_full, russell_X_valid_full, russell_y_train_full, russell_y_valid_full = train_test_split(russell_X_train_full,
                                                                                        russell_y_train_full,
                                                                                        stratify=russell_y_train_full,
                                                                                        test_size=0.1, shuffle=True)
sp_X_train_full, sp_X_test_full, sp_y_train_full, sp_y_test_full = train_test_split(sp_X_seq_flatten,
                                                                                        sp_y_seq,
                                                                                        stratify=sp_y_seq,
                                                                                        test_size=0.1, shuffle=True)
sp_X_train_full, sp_X_valid_full, sp_y_train_full, sp_y_valid_full = train_test_split(sp_X_train_full,
                                                                                        sp_y_train_full,
                                                                                        stratify=sp_y_train_full,
                                                                                        test_size=0.1, shuffle=True)

In [109]:
## pca features
dji_X_train_pca, dji_X_test_pca, dji_y_train_pca, dji_y_test_pca = train_test_split(dji_X_pca_seq_flatten,
                                                                                        dji_y_seq,
                                                                                        stratify=dji_y_seq,
                                                                                        test_size=0.1, shuffle=True)
dji_X_train_pca, dji_X_valid_pca, dji_y_train_pca, dji_y_valid_pca = train_test_split(dji_X_train_pca,
                                                                                        dji_y_train_pca,
                                                                                        stratify=dji_y_train_pca,
                                                                                        test_size=0.1, shuffle=True)
nasdaq_X_train_pca, nasdaq_X_test_pca, nasdaq_y_train_pca, nasdaq_y_test_pca = train_test_split(nasdaq_X_pca_seq_flatten,
                                                                                        nasdaq_y_seq,
                                                                                        stratify=nasdaq_y_seq,
                                                                                        test_size=0.1, shuffle=True)
nasdaq_X_train_pca, nasdaq_X_valid_pca, nasdaq_y_train_pca, nasdaq_y_valid_pca = train_test_split(nasdaq_X_train_pca,
                                                                                        nasdaq_y_train_pca,
                                                                                        stratify=nasdaq_y_train_pca,
                                                                                        test_size=0.1, shuffle=True)
nyse_X_train_pca, nyse_X_test_pca, nyse_y_train_pca, nyse_y_test_pca = train_test_split(nyse_X_pca_seq_flatten,
                                                                                        nyse_y_seq,
                                                                                        stratify=nyse_y_seq,
                                                                                        test_size=0.1, shuffle=True)
nyse_X_train_pca, nyse_X_valid_pca, nyse_y_train_pca, nyse_y_valid_pca = train_test_split(nyse_X_train_pca,
                                                                                        nyse_y_train_pca,
                                                                                        stratify=nyse_y_train_pca,
                                                                                        test_size=0.1, shuffle=True)
russell_X_train_pca, russell_X_test_pca, russell_y_train_pca, russell_y_test_pca = train_test_split(russell_X_pca_seq_flatten,
                                                                                        russell_y_seq,
                                                                                        stratify=russell_y_seq,
                                                                                        test_size=0.1, shuffle=True)
russell_X_train_pca, russell_X_valid_pca, russell_y_train_pca, russell_y_valid_pca = train_test_split(russell_X_train_pca,
                                                                                        russell_y_train_pca,
                                                                                        stratify=russell_y_train_pca,
                                                                                        test_size=0.1, shuffle=True)
sp_X_train_pca, sp_X_test_pca, sp_y_train_pca, sp_y_test_pca = train_test_split(sp_X_pca_seq_flatten,
                                                                                        sp_y_seq,
                                                                                        stratify=sp_y_seq,
                                                                                        test_size=0.1, shuffle=True)
sp_X_train_pca, sp_X_valid_pca, sp_y_train_pca, sp_y_valid_pca = train_test_split(sp_X_train_pca,
                                                                                        sp_y_train_pca,
                                                                                        stratify=sp_y_train_pca,
                                                                                        test_size=0.1, shuffle=True)

In [110]:
## ti features
dji_X_train_ti, dji_X_test_ti, dji_y_train_ti, dji_y_test_ti = train_test_split(dji_X_ti_seq_flatten,
                                                                                        dji_y_seq,
                                                                                        stratify=dji_y_seq,
                                                                                        test_size=0.1, shuffle=True)
dji_X_train_ti, dji_X_valid_ti, dji_y_train_ti, dji_y_valid_ti = train_test_split(dji_X_train_ti,
                                                                                        dji_y_train_ti,
                                                                                        stratify=dji_y_train_ti,
                                                                                        test_size=0.1, shuffle=True)
nasdaq_X_train_ti, nasdaq_X_test_ti, nasdaq_y_train_ti, nasdaq_y_test_ti = train_test_split(nasdaq_X_ti_seq_flatten,
                                                                                        nasdaq_y_seq,
                                                                                        stratify=nasdaq_y_seq,
                                                                                        test_size=0.1, shuffle=True)
nasdaq_X_train_ti, nasdaq_X_valid_ti, nasdaq_y_train_ti, nasdaq_y_valid_ti = train_test_split(nasdaq_X_train_ti,
                                                                                        nasdaq_y_train_ti,
                                                                                        stratify=nasdaq_y_train_ti,
                                                                                        test_size=0.1, shuffle=True)
nyse_X_train_ti, nyse_X_test_ti, nyse_y_train_ti, nyse_y_test_ti = train_test_split(nyse_X_ti_seq_flatten,
                                                                                        nyse_y_seq,
                                                                                        stratify=nyse_y_seq,
                                                                                        test_size=0.1, shuffle=True)
nyse_X_train_ti, nyse_X_valid_ti, nyse_y_train_ti, nyse_y_valid_ti = train_test_split(nyse_X_train_ti,
                                                                                        nyse_y_train_ti,
                                                                                        stratify=nyse_y_train_ti,
                                                                                        test_size=0.1, shuffle=True)
russell_X_train_ti, russell_X_test_ti, russell_y_train_ti, russell_y_test_ti = train_test_split(russell_X_ti_seq_flatten,
                                                                                        russell_y_seq,
                                                                                        stratify=russell_y_seq,
                                                                                        test_size=0.1, shuffle=True)
russell_X_train_ti, russell_X_valid_ti, russell_y_train_ti, russell_y_valid_ti = train_test_split(russell_X_train_ti,
                                                                                        russell_y_train_ti,
                                                                                        stratify=russell_y_train_ti,
                                                                                        test_size=0.1, shuffle=True)
sp_X_train_ti, sp_X_test_ti, sp_y_train_ti, sp_y_test_ti = train_test_split(sp_X_ti_seq_flatten,
                                                                                        sp_y_seq,
                                                                                        stratify=sp_y_seq,
                                                                                        test_size=0.1, shuffle=True)
sp_X_train_ti, sp_X_valid_ti, sp_y_train_ti, sp_y_valid_ti = train_test_split(sp_X_train_ti,
                                                                                        sp_y_train_ti,
                                                                                        stratify=sp_y_train_ti,
                                                                                        test_size=0.1, shuffle=True)

### Model training and evaluation

#### Full Features

In [111]:
## DJI
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam(lr=1e-2)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=50, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(dji_X_train_full, np.array(dji_y_train_full), 
          epochs=200, batch_size=32,
          validation_data=(dji_X_valid_full, np.array(dji_y_valid_full)), callbacks=[early_stopping])

Epoch 1/200
44/44 [==============================] - 0s 7ms/step - loss: 1.9173 - acc: 0.4953 - f1: 0.4693 - val_loss: 1.4425 - val_acc: 0.5000 - val_f1: 0.4882
Epoch 2/200
44/44 [==============================] - 0s 3ms/step - loss: 0.7450 - acc: 0.6875 - f1: 0.6783 - val_loss: 1.1536 - val_acc: 0.5192 - val_f1: 0.5013
Epoch 3/200
44/44 [==============================] - 0s 3ms/step - loss: 0.3782 - acc: 0.8523 - f1: 0.8468 - val_loss: 1.3342 - val_acc: 0.5064 - val_f1: 0.4987
Epoch 4/200
44/44 [==============================] - 0s 3ms/step - loss: 0.1381 - acc: 0.9577 - f1: 0.9559 - val_loss: 1.6427 - val_acc: 0.4359 - val_f1: 0.4314
Epoch 5/200
44/44 [==============================] - 0s 3ms/step - loss: 0.0665 - acc: 0.9806 - f1: 0.9797 - val_loss: 1.9414 - val_acc: 0.4423 - val_f1: 0.4388
Epoch 6/200
44/44 [==============================] - 0s 3ms/step - loss: 0.0284 - acc: 0.9935 - f1: 0.9934 - val_loss: 2.0400 - val_acc: 0.4423 - val_f1: 0.4298
Epoch 7/200
44/44 [===============

Epoch 51/200
44/44 [==============================] - 0s 3ms/step - loss: 5.4836e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 3.4003 - val_acc: 0.4423 - val_f1: 0.4351
Epoch 52/200
44/44 [==============================] - 0s 3ms/step - loss: 5.2474e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 3.4101 - val_acc: 0.4423 - val_f1: 0.4351


In [112]:
result_dji_full = model.predict(dji_X_test_full)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"DJI Accuracy: {accuracy_score(result_dji_full, dji_y_test_full)}")
print(f"DJI F1: {f1_score(result_dji_full, dji_y_test_full, average='macro')}")

DJI Accuracy: 0.5317919075144508
DJI F1: 0.5277525022747953


In [113]:
## NASDAQ
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(nasdaq_X_train_full, np.array(nasdaq_y_train_full), 
          epochs=200, batch_size=128,
          validation_data=(nasdaq_X_valid_full, np.array(nasdaq_y_valid_full)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 22ms/step - loss: 0.8546 - acc: 0.4946 - f1: 0.4549 - val_loss: 0.8546 - val_acc: 0.5128 - val_f1: 0.5342
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 0.4694 - acc: 0.7986 - f1: 0.7929 - val_loss: 0.8579 - val_acc: 0.4808 - val_f1: 0.4675
Epoch 3/200
11/11 [==============================] - 0s 5ms/step - loss: 0.2826 - acc: 0.9405 - f1: 0.9391 - val_loss: 0.9001 - val_acc: 0.4679 - val_f1: 0.4743
Epoch 4/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1621 - acc: 0.9842 - f1: 0.9833 - val_loss: 0.9680 - val_acc: 0.4936 - val_f1: 0.4665
Epoch 5/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0900 - acc: 0.9943 - f1: 0.9941 - val_loss: 1.0190 - val_acc: 0.5000 - val_f1: 0.4585
Epoch 6/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0512 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.0912 - val_acc: 0.5064 - val_f1: 0.5044
Epoch 7/200
11/11 [==============

11/11 [==============================] - 0s 5ms/step - loss: 9.5419e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.1652 - val_acc: 0.5321 - val_f1: 0.5387
Epoch 101/200
11/11 [==============================] - 0s 5ms/step - loss: 9.3398e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.1705 - val_acc: 0.5321 - val_f1: 0.5387


In [114]:
## NASDAQ
result_nasdaq_full = model.predict(nasdaq_X_test_full)
result_nasdaq_full = (result_nasdaq_full > 0.5).astype(int)
print(f"NASDAQ Accuracy: {accuracy_score(result_nasdaq_full, nasdaq_y_test_full)}")
print(f"NASDAQ F1: {f1_score(result_nasdaq_full, nasdaq_y_test_full, average='macro')}")

NASDAQ Accuracy: 0.5433526011560693
NASDAQ F1: 0.539412934317393


In [119]:
## NYSE
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(nyse_X_train_full, np.array(nyse_y_train_full), 
          epochs=200, batch_size=32,
          validation_data=(nyse_X_valid_full, np.array(nyse_y_valid_full)), callbacks=[early_stopping])

Epoch 1/200
44/44 [==============================] - 0s 7ms/step - loss: 0.9206 - acc: 0.4982 - f1: 0.4831 - val_loss: 0.7966 - val_acc: 0.4808 - val_f1: 0.4497
Epoch 2/200
44/44 [==============================] - 0s 3ms/step - loss: 0.4271 - acc: 0.8194 - f1: 0.8135 - val_loss: 0.8669 - val_acc: 0.5321 - val_f1: 0.4968
Epoch 3/200
44/44 [==============================] - 0s 2ms/step - loss: 0.2162 - acc: 0.9527 - f1: 0.9508 - val_loss: 0.9473 - val_acc: 0.5256 - val_f1: 0.5115
Epoch 4/200
44/44 [==============================] - 0s 3ms/step - loss: 0.0927 - acc: 0.9957 - f1: 0.9954 - val_loss: 1.0896 - val_acc: 0.4615 - val_f1: 0.4573
Epoch 5/200
44/44 [==============================] - 0s 3ms/step - loss: 0.0428 - acc: 0.9986 - f1: 0.9985 - val_loss: 1.1584 - val_acc: 0.4936 - val_f1: 0.4874
Epoch 6/200
44/44 [==============================] - 0s 3ms/step - loss: 0.0220 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.2535 - val_acc: 0.4872 - val_f1: 0.4829
Epoch 7/200
44/44 [===============

Epoch 51/200
44/44 [==============================] - 0s 3ms/step - loss: 1.4068e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.2646 - val_acc: 0.4808 - val_f1: 0.4739
Epoch 52/200
44/44 [==============================] - 0s 2ms/step - loss: 1.3409e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.2761 - val_acc: 0.4808 - val_f1: 0.4739
Epoch 53/200
44/44 [==============================] - 0s 3ms/step - loss: 1.2801e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.2865 - val_acc: 0.4872 - val_f1: 0.4793
Epoch 54/200
44/44 [==============================] - 0s 3ms/step - loss: 1.2234e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.2976 - val_acc: 0.4872 - val_f1: 0.4793
Epoch 55/200
44/44 [==============================] - 0s 3ms/step - loss: 1.1690e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.3054 - val_acc: 0.4872 - val_f1: 0.4793
Epoch 56/200
44/44 [==============================] - 0s 3ms/step - loss: 1.1169e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.3200 - val_acc: 0.4872 - val_f1: 0.4793
Epoc

In [120]:
## NYSE
result_nyse_full = model.predict(nyse_X_test_full)
result_nyse_full = (result_nyse_full > 0.5).astype(int)
print(f"NYSE Accuracy: {accuracy_score(result_nyse_full, nyse_y_test_full)}")
print(f"NYSE F1: {f1_score(result_nyse_full, nyse_y_test_full, average='macro')}")

NYSE Accuracy: 0.42196531791907516
NYSE F1: 0.4219460037423149


In [135]:
## Russell
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=50, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(russell_X_train_full, np.array(russell_y_train_full), 
          epochs=200, batch_size=128,
          validation_data=(russell_X_valid_full, np.array(russell_y_valid_full)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 23ms/step - loss: 0.7866 - acc: 0.5082 - f1: 0.4990 - val_loss: 0.7876 - val_acc: 0.5128 - val_f1: 0.4499
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 0.4682 - acc: 0.7921 - f1: 0.7901 - val_loss: 0.8128 - val_acc: 0.4487 - val_f1: 0.3982
Epoch 3/200
11/11 [==============================] - 0s 5ms/step - loss: 0.3086 - acc: 0.9125 - f1: 0.9116 - val_loss: 0.8476 - val_acc: 0.4936 - val_f1: 0.4837
Epoch 4/200
11/11 [==============================] - 0s 6ms/step - loss: 0.1949 - acc: 0.9692 - f1: 0.9686 - val_loss: 0.8978 - val_acc: 0.5000 - val_f1: 0.4829
Epoch 5/200
11/11 [==============================] - 0s 6ms/step - loss: 0.1255 - acc: 0.9878 - f1: 0.9877 - val_loss: 0.9500 - val_acc: 0.4872 - val_f1: 0.4461
Epoch 6/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0733 - acc: 0.9971 - f1: 0.9971 - val_loss: 1.0362 - val_acc: 0.5128 - val_f1: 0.5341
Epoch 7/200
11/11 [==============

In [136]:
## russell
result_russell_full = model.predict(nyse_X_test_full)
result_russell_full = (result_russell_full > 0.5).astype(int)
print(f"russell Accuracy: {accuracy_score(result_russell_full, russell_y_test_full)}")
print(f"russell F1: {f1_score(result_russell_full, russell_y_test_full, average='macro')}")

russell Accuracy: 0.4393063583815029
russell F1: 0.43863111765296225


In [131]:
## S&P 500
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(sp_X_train_full, np.array(sp_y_train_full), 
          epochs=200, batch_size=128,
          validation_data=(sp_X_valid_full, np.array(sp_y_valid_full)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 23ms/step - loss: 0.7596 - acc: 0.5011 - f1: 0.4828 - val_loss: 0.7156 - val_acc: 0.5897 - val_f1: 0.5653
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 0.4650 - acc: 0.8108 - f1: 0.8056 - val_loss: 0.7784 - val_acc: 0.5385 - val_f1: 0.5112
Epoch 3/200
11/11 [==============================] - 0s 5ms/step - loss: 0.2956 - acc: 0.9333 - f1: 0.9318 - val_loss: 0.7869 - val_acc: 0.5321 - val_f1: 0.5106
Epoch 4/200
11/11 [==============================] - 0s 6ms/step - loss: 0.1814 - acc: 0.9814 - f1: 0.9810 - val_loss: 0.8414 - val_acc: 0.5192 - val_f1: 0.5011
Epoch 5/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1096 - acc: 0.9978 - f1: 0.9979 - val_loss: 0.8730 - val_acc: 0.4936 - val_f1: 0.4500
Epoch 6/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0655 - acc: 0.9986 - f1: 0.9986 - val_loss: 0.9311 - val_acc: 0.4936 - val_f1: 0.4490
Epoch 7/200
11/11 [==============

11/11 [==============================] - 0s 6ms/step - loss: 6.6040e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.0827 - val_acc: 0.5385 - val_f1: 0.5002
Epoch 101/200
11/11 [==============================] - 0s 5ms/step - loss: 6.4539e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.0876 - val_acc: 0.5385 - val_f1: 0.5002


In [74]:
## russell
result_sp_full = model.predict(nyse_X_test_full)
result_sp_full = (result_sp_full > 0.5).astype(int)
print(f"NYSE Accuracy: {accuracy_score(result_sp_full, sp_y_test_full)}")
print(f"NYSE F1: {f1_score(result_sp_full, sp_y_test_full, average='macro')}")

NYSE Accuracy: 0.5086705202312138
NYSE F1: 0.5062951149907672


#### PCA Features

In [137]:
## DJI
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam(lr=1e-2)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(dji_X_train_pca, np.array(dji_y_train_pca), 
          epochs=200, batch_size=128,
          validation_data=(dji_X_valid_pca, np.array(dji_y_valid_pca)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 20ms/step - loss: 1.0717 - acc: 0.5283 - f1: 0.5187 - val_loss: 0.8373 - val_acc: 0.5256 - val_f1: 0.5212
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5342 - acc: 0.7520 - f1: 0.7511 - val_loss: 0.8973 - val_acc: 0.5705 - val_f1: 0.5524
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 0.2628 - acc: 0.9054 - f1: 0.9041 - val_loss: 0.9687 - val_acc: 0.5128 - val_f1: 0.4518
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 0.1100 - acc: 0.9685 - f1: 0.9683 - val_loss: 1.2015 - val_acc: 0.5256 - val_f1: 0.4887
Epoch 5/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0538 - acc: 0.9892 - f1: 0.9891 - val_loss: 1.1073 - val_acc: 0.5705 - val_f1: 0.5185
Epoch 6/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0178 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.2855 - val_acc: 0.5192 - val_f1: 0.4649
Epoch 7/200
11/11 [==============

Epoch 51/200
11/11 [==============================] - 0s 4ms/step - loss: 7.8241e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.8456 - val_acc: 0.5321 - val_f1: 0.4237
Epoch 52/200
11/11 [==============================] - 0s 4ms/step - loss: 7.5549e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.8513 - val_acc: 0.5321 - val_f1: 0.4237
Epoch 53/200
11/11 [==============================] - 0s 4ms/step - loss: 7.3036e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.8560 - val_acc: 0.5321 - val_f1: 0.4237
Epoch 54/200
11/11 [==============================] - 0s 4ms/step - loss: 7.0563e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.8611 - val_acc: 0.5321 - val_f1: 0.4237
Epoch 55/200
11/11 [==============================] - 0s 4ms/step - loss: 6.8248e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.8666 - val_acc: 0.5385 - val_f1: 0.4282
Epoch 56/200
11/11 [==============================] - 0s 4ms/step - loss: 6.6033e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.8714 - val_acc: 0.5385 - val_f1: 0.4282
Epoc

In [138]:
result_dji_full = model.predict(dji_X_test_pca)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"DJI Accuracy: {accuracy_score(result_dji_full, dji_y_test_pca)}")
print(f"DJI F1: {f1_score(result_dji_full, dji_y_test_pca, average='macro')}")

DJI Accuracy: 0.5202312138728323
DJI F1: 0.5179117005203961


In [139]:
## NASDAQ
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(nasdaq_X_train_pca, np.array(nasdaq_y_train_pca), 
          epochs=200, batch_size=128,
          validation_data=(nasdaq_X_valid_pca, np.array(nasdaq_y_valid_pca)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 21ms/step - loss: 0.8331 - acc: 0.5054 - f1: 0.4930 - val_loss: 0.7909 - val_acc: 0.5192 - val_f1: 0.5025
Epoch 2/200
11/11 [==============================] - 0s 4ms/step - loss: 0.4516 - acc: 0.8280 - f1: 0.8230 - val_loss: 0.7716 - val_acc: 0.5449 - val_f1: 0.4880
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 0.2944 - acc: 0.9448 - f1: 0.9440 - val_loss: 0.7870 - val_acc: 0.5641 - val_f1: 0.5444
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 0.1919 - acc: 0.9849 - f1: 0.9847 - val_loss: 0.8389 - val_acc: 0.5256 - val_f1: 0.4887
Epoch 5/200
11/11 [==============================] - 0s 5ms/step - loss: 0.1254 - acc: 0.9957 - f1: 0.9956 - val_loss: 0.8499 - val_acc: 0.5769 - val_f1: 0.5544
Epoch 6/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0813 - acc: 0.9993 - f1: 0.9993 - val_loss: 0.9049 - val_acc: 0.5705 - val_f1: 0.5500
Epoch 7/200
11/11 [==============

11/11 [==============================] - 0s 4ms/step - loss: 1.0309e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.9815 - val_acc: 0.5577 - val_f1: 0.5420
Epoch 101/200
11/11 [==============================] - 0s 4ms/step - loss: 1.0085e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.9864 - val_acc: 0.5577 - val_f1: 0.5420
Epoch 102/200
11/11 [==============================] - 0s 4ms/step - loss: 9.8714e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 1.9902 - val_acc: 0.5577 - val_f1: 0.5420


In [140]:
## NASDAQ
result_nasdaq_full = model.predict(nasdaq_X_test_pca)
result_nasdaq_full = (result_nasdaq_full > 0.5).astype(int)
print(f"NASDAQ Accuracy: {accuracy_score(result_nasdaq_full, nasdaq_y_test_pca)}")
print(f"NASDAQ F1: {f1_score(result_nasdaq_full, nasdaq_y_test_pca, average='macro')}")

NASDAQ Accuracy: 0.5317919075144508
NASDAQ F1: 0.526667792602601


In [141]:
## NYSE
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(nyse_X_train_pca, np.array(nyse_y_train_pca), 
          epochs=200, batch_size=128,
          validation_data=(nyse_X_valid_pca, np.array(nyse_y_valid_pca)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 21ms/step - loss: 0.8535 - acc: 0.4824 - f1: 0.4686 - val_loss: 0.7575 - val_acc: 0.5000 - val_f1: 0.5059
Epoch 2/200
11/11 [==============================] - 0s 4ms/step - loss: 0.4948 - acc: 0.7756 - f1: 0.7694 - val_loss: 0.7460 - val_acc: 0.5705 - val_f1: 0.5007
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 0.3508 - acc: 0.9197 - f1: 0.9189 - val_loss: 0.7694 - val_acc: 0.5192 - val_f1: 0.4762
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 0.2467 - acc: 0.9763 - f1: 0.9760 - val_loss: 0.8121 - val_acc: 0.5385 - val_f1: 0.4755
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1749 - acc: 0.9907 - f1: 0.9905 - val_loss: 0.8438 - val_acc: 0.5321 - val_f1: 0.4806
Epoch 6/200
11/11 [==============================] - 0s 4ms/step - loss: 0.1206 - acc: 0.9957 - f1: 0.9955 - val_loss: 0.8992 - val_acc: 0.5513 - val_f1: 0.5251
Epoch 7/200
11/11 [==============

11/11 [==============================] - 0s 4ms/step - loss: 8.8118e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.4851 - val_acc: 0.5128 - val_f1: 0.4916
Epoch 101/200
11/11 [==============================] - 0s 4ms/step - loss: 8.6154e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.4908 - val_acc: 0.5128 - val_f1: 0.4916
Epoch 102/200
11/11 [==============================] - 0s 4ms/step - loss: 8.4249e-05 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.4969 - val_acc: 0.5128 - val_f1: 0.4916


In [142]:
## NYSE
result_nyse_full = model.predict(nyse_X_test_pca)
result_nyse_full = (result_nyse_full > 0.5).astype(int)
print(f"NYSE Accuracy: {accuracy_score(result_nyse_full, nyse_y_test_pca)}")
print(f"NYSE F1: {f1_score(result_nyse_full, nyse_y_test_pca, average='macro')}")

NYSE Accuracy: 0.5028901734104047
NYSE F1: 0.4968208874458875


In [143]:
## Russell
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(russell_X_train_pca, np.array(russell_y_train_pca), 
          epochs=200, batch_size=128,
          validation_data=(russell_X_valid_pca, np.array(russell_y_valid_pca)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 22ms/step - loss: 0.8235 - acc: 0.4953 - f1: 0.4890 - val_loss: 0.7499 - val_acc: 0.4936 - val_f1: 0.4839
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 0.4578 - acc: 0.8136 - f1: 0.8082 - val_loss: 0.7829 - val_acc: 0.5192 - val_f1: 0.5512
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 0.3044 - acc: 0.9462 - f1: 0.9457 - val_loss: 0.8279 - val_acc: 0.5128 - val_f1: 0.5641
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 0.2055 - acc: 0.9806 - f1: 0.9805 - val_loss: 0.8307 - val_acc: 0.5513 - val_f1: 0.5542
Epoch 5/200
11/11 [==============================] - 0s 4ms/step - loss: 0.1366 - acc: 0.9993 - f1: 0.9993 - val_loss: 0.8991 - val_acc: 0.5385 - val_f1: 0.5504
Epoch 6/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0907 - acc: 1.0000 - f1: 1.0000 - val_loss: 0.9424 - val_acc: 0.5577 - val_f1: 0.5691
Epoch 7/200
11/11 [==============

11/11 [==============================] - 0s 4ms/step - loss: 1.1853e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.2073 - val_acc: 0.5128 - val_f1: 0.5341
Epoch 101/200
11/11 [==============================] - 0s 4ms/step - loss: 1.1595e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.2140 - val_acc: 0.5128 - val_f1: 0.5341


In [144]:
## russell
result_russell_full = model.predict(russell_X_test_pca)
result_russell_full = (result_russell_full > 0.5).astype(int)
print(f"russell Accuracy: {accuracy_score(result_russell_full, russell_y_test_pca)}")
print(f"russell F1: {f1_score(result_russell_full, russell_y_test_pca, average='macro')}")

russell Accuracy: 0.4913294797687861
russell F1: 0.48844086021505373


In [145]:
## S&P 500
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(sp_X_train_pca, np.array(sp_y_train_pca), 
          epochs=200, batch_size=128,
          validation_data=(sp_X_valid_pca, np.array(sp_y_valid_pca)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 21ms/step - loss: 0.8495 - acc: 0.4918 - f1: 0.4757 - val_loss: 0.8103 - val_acc: 0.5128 - val_f1: 0.4408
Epoch 2/200
11/11 [==============================] - 0s 4ms/step - loss: 0.4700 - acc: 0.7885 - f1: 0.7808 - val_loss: 0.8231 - val_acc: 0.4872 - val_f1: 0.4438
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 0.3208 - acc: 0.9398 - f1: 0.9389 - val_loss: 0.8480 - val_acc: 0.4808 - val_f1: 0.3858
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 0.2256 - acc: 0.9728 - f1: 0.9723 - val_loss: 0.9017 - val_acc: 0.4872 - val_f1: 0.3758
Epoch 5/200
11/11 [==============================] - 0s 4ms/step - loss: 0.1608 - acc: 0.9871 - f1: 0.9869 - val_loss: 0.9337 - val_acc: 0.5064 - val_f1: 0.4098
Epoch 6/200
11/11 [==============================] - 0s 4ms/step - loss: 0.1119 - acc: 0.9971 - f1: 0.9971 - val_loss: 0.9865 - val_acc: 0.4679 - val_f1: 0.3520
Epoch 7/200
11/11 [==============

11/11 [==============================] - 0s 4ms/step - loss: 1.1857e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.0756 - val_acc: 0.5705 - val_f1: 0.4647
Epoch 101/200
11/11 [==============================] - 0s 4ms/step - loss: 1.1608e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.0808 - val_acc: 0.5705 - val_f1: 0.4647


In [146]:
## russell
result_sp_full = model.predict(sp_X_test_pca)
result_sp_full = (result_sp_full > 0.5).astype(int)
print(f"S&P 500 Accuracy: {accuracy_score(result_sp_full, sp_y_test_pca)}")
print(f"S&P 500 F1: {f1_score(result_sp_full, sp_y_test_pca, average='macro')}")

S&P 500 Accuracy: 0.4508670520231214
S&P 500 F1: 0.44485728762033444


#### Technical indicator features

In [101]:
## DJI
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam(lr=1e-2)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(dji_X_train_ti, np.array(dji_y_train_ti), 
          epochs=200, batch_size=128,
          validation_data=(dji_X_valid_ti, np.array(dji_y_valid_ti)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 18ms/step - loss: 1.1505 - acc: 0.5176 - f1: 0.4820 - val_loss: 0.9211 - val_acc: 0.4872 - val_f1: 0.5011
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 0.7437 - acc: 0.5742 - f1: 0.5586 - val_loss: 0.8446 - val_acc: 0.4872 - val_f1: 0.4431
Epoch 3/200
11/11 [==============================] - 0s 3ms/step - loss: 0.6493 - acc: 0.6473 - f1: 0.6386 - val_loss: 0.8815 - val_acc: 0.4936 - val_f1: 0.4183
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 0.5585 - acc: 0.6975 - f1: 0.6848 - val_loss: 0.9150 - val_acc: 0.4808 - val_f1: 0.4871
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 0.4748 - acc: 0.7692 - f1: 0.7646 - val_loss: 0.8535 - val_acc: 0.5513 - val_f1: 0.5196
Epoch 6/200
11/11 [==============================] - 0s 3ms/step - loss: 0.4142 - acc: 0.8136 - f1: 0.8100 - val_loss: 0.9378 - val_acc: 0.5256 - val_f1: 0.5183
Epoch 7/200
11/11 [==============

11/11 [==============================] - 0s 3ms/step - loss: 2.0332e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 4.9771 - val_acc: 0.5256 - val_f1: 0.4878
Epoch 101/200
11/11 [==============================] - 0s 3ms/step - loss: 1.9792e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 4.9855 - val_acc: 0.5256 - val_f1: 0.4878
Epoch 102/200
11/11 [==============================] - 0s 3ms/step - loss: 1.9474e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 4.9954 - val_acc: 0.5256 - val_f1: 0.4878


In [102]:
result_dji_full = model.predict(dji_X_test_ti)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"DJI Accuracy: {accuracy_score(result_dji_full, dji_y_test_ti)}")
print(f"DJI F1: {f1_score(result_dji_full, dji_y_test_ti, average='macro')}")

DJI Accuracy: 0.4913294797687861
DJI F1: 0.48636977058029696


In [103]:
## NASDAQ
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(nasdaq_X_train_ti, np.array(nasdaq_y_train_ti), 
          epochs=200, batch_size=128,
          validation_data=(nasdaq_X_valid_ti, np.array(nasdaq_y_valid_ti)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 18ms/step - loss: 0.7580 - acc: 0.4925 - f1: 0.4767 - val_loss: 0.7504 - val_acc: 0.4936 - val_f1: 0.4437
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 0.6375 - acc: 0.6308 - f1: 0.6126 - val_loss: 0.7352 - val_acc: 0.5192 - val_f1: 0.5471
Epoch 3/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5740 - acc: 0.7233 - f1: 0.7052 - val_loss: 0.7403 - val_acc: 0.5128 - val_f1: 0.5328
Epoch 4/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5295 - acc: 0.7785 - f1: 0.7706 - val_loss: 0.7503 - val_acc: 0.4872 - val_f1: 0.5253
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 0.4935 - acc: 0.8086 - f1: 0.8006 - val_loss: 0.7669 - val_acc: 0.5064 - val_f1: 0.5519
Epoch 6/200
11/11 [==============================] - 0s 3ms/step - loss: 0.4578 - acc: 0.8473 - f1: 0.8422 - val_loss: 0.7701 - val_acc: 0.5192 - val_f1: 0.5578
Epoch 7/200
11/11 [==============

Epoch 102/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0012 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.6178 - val_acc: 0.5192 - val_f1: 0.5498


In [104]:
## NASDAQ
result_nasdaq_full = model.predict(nasdaq_X_test_ti)
result_nasdaq_full = (result_nasdaq_full > 0.5).astype(int)
print(f"NASDAQ Accuracy: {accuracy_score(result_nasdaq_full, nasdaq_y_test_ti)}")
print(f"NASDAQ F1: {f1_score(result_nasdaq_full, nasdaq_y_test_ti, average='macro')}")

NASDAQ Accuracy: 0.5144508670520231
NASDAQ F1: 0.5040950040950041


In [115]:
## NYSE
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(nyse_X_train_ti, np.array(nyse_y_train_ti), 
          epochs=200, batch_size=128,
          validation_data=(nyse_X_valid_ti, np.array(nyse_y_valid_ti)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 21ms/step - loss: 0.7429 - acc: 0.4996 - f1: 0.4900 - val_loss: 0.7443 - val_acc: 0.5321 - val_f1: 0.5311
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 0.6469 - acc: 0.6222 - f1: 0.6039 - val_loss: 0.7664 - val_acc: 0.5128 - val_f1: 0.4233
Epoch 3/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5876 - acc: 0.6975 - f1: 0.6921 - val_loss: 0.7496 - val_acc: 0.4487 - val_f1: 0.3947
Epoch 4/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5474 - acc: 0.7541 - f1: 0.7501 - val_loss: 0.7609 - val_acc: 0.5000 - val_f1: 0.4908
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5055 - acc: 0.8050 - f1: 0.7997 - val_loss: 0.7782 - val_acc: 0.4615 - val_f1: 0.4426
Epoch 6/200
11/11 [==============================] - 0s 3ms/step - loss: 0.4760 - acc: 0.8251 - f1: 0.8230 - val_loss: 0.7855 - val_acc: 0.5064 - val_f1: 0.4884
Epoch 7/200
11/11 [==============

In [116]:
## NYSE
result_nyse_full = model.predict(nyse_X_test_ti)
result_nyse_full = (result_nyse_full > 0.5).astype(int)
print(f"NYSE Accuracy: {accuracy_score(result_nyse_full, nyse_y_test_ti)}")
print(f"NYSE F1: {f1_score(result_nyse_full, nyse_y_test_ti, average='macro')}")

NYSE Accuracy: 0.5028901734104047
NYSE F1: 0.4939455782312925


In [135]:
## Russell
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(russell_X_train_ti, np.array(russell_y_train_ti), 
          epochs=200, batch_size=128,
          validation_data=(russell_X_valid_ti, np.array(russell_y_valid_ti)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 19ms/step - loss: 0.7591 - acc: 0.4989 - f1: 0.4672 - val_loss: 0.7241 - val_acc: 0.4551 - val_f1: 0.3734
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 0.6651 - acc: 0.5935 - f1: 0.5826 - val_loss: 0.7084 - val_acc: 0.4744 - val_f1: 0.4500
Epoch 3/200
11/11 [==============================] - 0s 3ms/step - loss: 0.6151 - acc: 0.6767 - f1: 0.6641 - val_loss: 0.7017 - val_acc: 0.5321 - val_f1: 0.4540
Epoch 4/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5768 - acc: 0.7305 - f1: 0.7261 - val_loss: 0.7265 - val_acc: 0.5000 - val_f1: 0.4354
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5430 - acc: 0.7706 - f1: 0.7668 - val_loss: 0.7363 - val_acc: 0.4808 - val_f1: 0.4518
Epoch 6/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5088 - acc: 0.7892 - f1: 0.7865 - val_loss: 0.7453 - val_acc: 0.5064 - val_f1: 0.4647
Epoch 7/200
11/11 [==============

Epoch 102/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0011 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.8076 - val_acc: 0.5385 - val_f1: 0.5510
Epoch 103/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0011 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.8139 - val_acc: 0.5385 - val_f1: 0.5510


In [136]:
## russell
result_russell_full = model.predict(russell_X_test_ti)
result_russell_full = (result_russell_full > 0.5).astype(int)
print(f"russell Accuracy: {accuracy_score(result_russell_full, russell_y_test_ti)}")
print(f"russell F1: {f1_score(result_russell_full, russell_y_test_ti, average='macro')}")

russell Accuracy: 0.5202312138728323
russell F1: 0.5201670843776107


In [139]:
## S&P 500
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

optimizer = keras.optimizers.Adam()
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit(sp_X_train_ti, np.array(sp_y_train_ti), 
          epochs=200, batch_size=128,
          validation_data=(sp_X_valid_ti, np.array(sp_y_valid_ti)), callbacks=[early_stopping])

Epoch 1/200
11/11 [==============================] - 0s 18ms/step - loss: 0.7261 - acc: 0.5398 - f1: 0.5277 - val_loss: 0.7596 - val_acc: 0.5192 - val_f1: 0.4555
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 0.6375 - acc: 0.6366 - f1: 0.6242 - val_loss: 0.7453 - val_acc: 0.4808 - val_f1: 0.4085
Epoch 3/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5810 - acc: 0.7047 - f1: 0.6900 - val_loss: 0.7218 - val_acc: 0.4872 - val_f1: 0.5202
Epoch 4/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5407 - acc: 0.7491 - f1: 0.7457 - val_loss: 0.7554 - val_acc: 0.5064 - val_f1: 0.4337
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5056 - acc: 0.7699 - f1: 0.7629 - val_loss: 0.7594 - val_acc: 0.4936 - val_f1: 0.4613
Epoch 6/200
11/11 [==============================] - 0s 4ms/step - loss: 0.4706 - acc: 0.8143 - f1: 0.8111 - val_loss: 0.7631 - val_acc: 0.5064 - val_f1: 0.4591
Epoch 7/200
11/11 [==============

11/11 [==============================] - 0s 3ms/step - loss: 9.8217e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.8724 - val_acc: 0.5064 - val_f1: 0.4500
Epoch 102/200
11/11 [==============================] - 0s 3ms/step - loss: 9.5450e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.8850 - val_acc: 0.5000 - val_f1: 0.4464
Epoch 103/200
11/11 [==============================] - 0s 3ms/step - loss: 9.3533e-04 - acc: 1.0000 - f1: 1.0000 - val_loss: 2.8917 - val_acc: 0.5256 - val_f1: 0.4911


In [142]:
## russell
result_sp_full = model.predict(sp_X_test_ti)
result_sp_full = (result_sp_full > 0.5).astype(int)
print(f"S&P Accuracy: {accuracy_score(result_sp_full, sp_y_test_ti)}")
print(f"S&P F1: {f1_score(result_sp_full, sp_y_test_ti, average='macro')}")

S&P Accuracy: 0.5202312138728323
S&P F1: 0.5108166115899567


## 3D CNN Pred

#### Full features

In [17]:
### Clean dataframe (full features)
dji_df_full = dji_df_full
nasdaq_df_full = nasdaq_df_full
nyse_df_full = nyse_df_full
russell_df_full = russell_df_full
sp_df_full = sp_df_full

## Combine
X_full = dji_df_full.drop(columns=["MOVEMENT"]).copy()
y_full = list(dji_df_full["MOVEMENT"].copy())

for df in [nasdaq_df_full, nyse_df_full, russell_df_full, sp_df_full]:
    X_full = np.concatenate((X_full, df.drop(columns=["MOVEMENT"]).copy()))
    y_full.extend(list(df["MOVEMENT"].copy()))

In [18]:
## Reshape
X_full = X_full.reshape((5, -1, 82))
y_full = np.array(y_full)

## Sequencing
X_seq, y_seq = generate_sequential_data_3d(X_full, y_full, 60)

In [19]:
X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(
    X_seq, y_seq, stratify=None, test_size=0.1, shuffle=False
)
X_seq_train, X_seq_valid, y_seq_train, y_seq_valid = train_test_split(
    X_seq_train, y_seq_train, stratify=None, test_size=0.1, shuffle=False
)

In [22]:
model_full = cnnpred_3d(5, 60, 82, [8, 8, 8])
epochs = 100
batch_size=128

model_full.compile(optimizer="Adam", loss="mae", 
                   metrics=["acc", f1])
model_full.fit(X_seq_train, y_seq_train, epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_seq_valid, y_seq_valid))

Epoch 1/100
11/11 [==============================] - 0s 32ms/step - loss: 0.4908 - acc: 0.5233 - f1: 0.4249 - val_loss: 0.4905 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 2/100
11/11 [==============================] - 0s 14ms/step - loss: 0.4762 - acc: 0.5369 - f1: 0.3508 - val_loss: 0.4827 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 3/100
11/11 [==============================] - 0s 14ms/step - loss: 0.4679 - acc: 0.5362 - f1: 0.3485 - val_loss: 0.4787 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 4/100
11/11 [==============================] - 0s 14ms/step - loss: 0.4651 - acc: 0.5362 - f1: 0.3484 - val_loss: 0.4771 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 5/100
11/11 [==============================] - 0s 14ms/step - loss: 0.4642 - acc: 0.5362 - f1: 0.3488 - val_loss: 0.4765 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 6/100
11/11 [==============================] - 0s 14ms/step - loss: 0.4634 - acc: 0.5362 - f1: 0.3485 - val_loss: 0.4763 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 7/100
11/11 [=========

In [23]:
result_full = model_full.predict(X_seq_test)
result_full = (result_full > 0.5).astype(int)
print(f"FULL Accuracy: {accuracy_score(result_full, y_seq_test)}")
print(f"FULL F1: {f1_score(result_full, y_seq_test, average='macro')}")

FULL Accuracy: 0.5953757225433526
FULL F1: 0.5534660766961652


#### PCA features

In [86]:
### Clean dataframe (full features)
dji_df_full = dji_df_pca
nasdaq_df_full = nasdaq_df_pca
nyse_df_full = nyse_df_pca
russell_df_full = russell_df_pca
sp_df_full = sp_df_pca

## Combine
X_full = dji_df_full.drop(columns=["MOVEMENT"]).copy()
y_full = list(dji_df_full["MOVEMENT"].copy())

for df in [nasdaq_df_full, nyse_df_full, russell_df_full, sp_df_full]:
    X_full = np.concatenate((X_full, df.drop(columns=["MOVEMENT"]).copy()))
    y_full.extend(list(df["MOVEMENT"].copy()))

In [87]:
## Reshape
X_full = X_full.reshape((5, -1, 30))
y_full = np.array(y_full)

## Sequencing
X_seq, y_seq = generate_sequential_data_3d(X_full, y_full, 60)

In [88]:
X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(
    X_seq, y_seq, stratify=None, test_size=0.1, shuffle=False
)
X_seq_train, X_seq_valid, y_seq_train, y_seq_valid = train_test_split(
    X_seq_train, y_seq_train, stratify=None, test_size=0.1, shuffle=False
)

In [89]:
model_full = cnnpred_3d(5, 60, 30, [8, 8, 8])
epochs = 100
batch_size=128

model_full.compile(optimizer="Adam", loss="mae", 
                   metrics=["acc", f1])
model_full.fit(X_seq_train, y_seq_train, epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_seq_valid, y_seq_valid))

Epoch 1/100
11/11 [==============================] - 0s 32ms/step - loss: 0.4721 - acc: 0.5384 - f1: 0.3675 - val_loss: 0.4720 - val_acc: 0.5192 - val_f1: 0.3653
Epoch 2/100
11/11 [==============================] - 0s 11ms/step - loss: 0.4661 - acc: 0.5376 - f1: 0.3567 - val_loss: 0.4729 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 3/100
11/11 [==============================] - 0s 10ms/step - loss: 0.4631 - acc: 0.5362 - f1: 0.3504 - val_loss: 0.4729 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 4/100
11/11 [==============================] - 0s 11ms/step - loss: 0.4621 - acc: 0.5362 - f1: 0.3500 - val_loss: 0.4726 - val_acc: 0.5256 - val_f1: 0.3670
Epoch 5/100
11/11 [==============================] - 0s 10ms/step - loss: 0.4570 - acc: 0.5398 - f1: 0.3588 - val_loss: 0.4705 - val_acc: 0.5192 - val_f1: 0.3653
Epoch 6/100
11/11 [==============================] - 0s 15ms/step - loss: 0.4545 - acc: 0.5427 - f1: 0.3683 - val_loss: 0.4681 - val_acc: 0.5256 - val_f1: 0.3736
Epoch 7/100
11/11 [=========

In [90]:
result_full = model_full.predict(X_seq_test)
result_full = (result_full > 0.5).astype(int)
print(f"FULL Accuracy: {accuracy_score(result_full, y_seq_test)}")
print(f"FULL F1: {f1_score(result_full, y_seq_test, average='macro')}")

FULL Accuracy: 0.6011560693641619
FULL F1: 0.45315864217325574


#### Technical indicator features

In [91]:
### Clean dataframe (full features)
dji_df_full = dji_df_ti
nasdaq_df_full = nasdaq_df_ti
nyse_df_full = nyse_df_ti
russell_df_full = russell_df_ti
sp_df_full = sp_df_ti

## Combine
X_full = dji_df_full.drop(columns=["MOVEMENT"]).copy()
y_full = list(dji_df_full["MOVEMENT"].copy())

for df in [nasdaq_df_full, nyse_df_full, russell_df_full, sp_df_full]:
    X_full = np.concatenate((X_full, df.drop(columns=["MOVEMENT"]).copy()))
    y_full.extend(list(df["MOVEMENT"].copy()))

In [92]:
## Reshape
X_full = X_full.reshape((5, -1, 13))
y_full = np.array(y_full)

## Sequencing
X_seq, y_seq = generate_sequential_data_3d(X_full, y_full, 60)

In [93]:
X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(
    X_seq, y_seq, stratify=None, test_size=0.1, shuffle=False
)
X_seq_train, X_seq_valid, y_seq_train, y_seq_valid = train_test_split(
    X_seq_train, y_seq_train, stratify=None, test_size=0.1, shuffle=False
)

In [94]:
model_full = cnnpred_3d(5, 60, 13, [8, 8, 8])
epochs = 100
batch_size=128

model_full.compile(optimizer="Adam", loss="mae", 
                   metrics=["acc", f1])
model_full.fit(X_seq_train, y_seq_train, epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_seq_valid, y_seq_valid))

Epoch 1/100
11/11 [==============================] - 0s 28ms/step - loss: 0.5185 - acc: 0.4616 - f1: 0.3163 - val_loss: 0.5053 - val_acc: 0.4744 - val_f1: 0.2900
Epoch 2/100
11/11 [==============================] - 0s 9ms/step - loss: 0.5088 - acc: 0.4566 - f1: 0.3643 - val_loss: 0.5012 - val_acc: 0.4808 - val_f1: 0.3560
Epoch 3/100
11/11 [==============================] - 0s 10ms/step - loss: 0.4998 - acc: 0.5061 - f1: 0.4976 - val_loss: 0.4980 - val_acc: 0.5449 - val_f1: 0.5569
Epoch 4/100
11/11 [==============================] - 0s 9ms/step - loss: 0.4938 - acc: 0.5276 - f1: 0.4894 - val_loss: 0.4977 - val_acc: 0.5256 - val_f1: 0.5358
Epoch 5/100
11/11 [==============================] - 0s 10ms/step - loss: 0.4894 - acc: 0.5283 - f1: 0.4351 - val_loss: 0.4974 - val_acc: 0.5064 - val_f1: 0.4887
Epoch 6/100
11/11 [==============================] - 0s 10ms/step - loss: 0.4833 - acc: 0.5333 - f1: 0.3998 - val_loss: 0.4955 - val_acc: 0.5449 - val_f1: 0.4697
Epoch 7/100
11/11 [===========

In [95]:
result_full = model_full.predict(X_seq_test)
result_full = (result_full > 0.5).astype(int)
print(f"FULL Accuracy: {accuracy_score(result_full, y_seq_test)}")
print(f"FULL F1: {f1_score(result_full, y_seq_test, average='macro')}")

FULL Accuracy: 0.47398843930635837
FULL F1: 0.46366640547814536


## LSTM

#### Full features

In [7]:
### DJI
# Split
X = np.array(dji_df_full.drop(columns=["MOVEMENT"]).copy())
y = np.array(dji_df_full["MOVEMENT"].copy())

In [8]:
# Split train test, should not shuffle as the data is time series
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=False
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, shuffle=False
)

In [9]:
# Get input ready for lstm
win_length = 60
batch_size = 32
num_features = 82
train_generator = TimeseriesGenerator(np.array(X_train), np.array(y_train),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

test_generator = TimeseriesGenerator(np.array(X_test), np.array(y_test),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

valid_generator = TimeseriesGenerator(np.array(X_valid), np.array(y_valid),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

In [10]:
model = lstm(win_length, num_features)

optimizer = keras.optimizers.Adam()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                 patience=20,
                                                 mode="min")
model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit_generator(train_generator,
                   epochs=100,
                   validation_data=valid_generator,
                   shuffle=False, callbacks=[early_stopping])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
44/44 [==============================] - 3s 70ms/step - loss: 0.7008 - acc: 0.5004 - f1: 0.4135 - val_loss: 0.7362 - val_acc: 0.4158 - val_f1: 0.3367
Epoch 2/50
44/44 [==============================] - 2s 55ms/step - loss: 0.6906 - acc: 0.5423 - f1: 0.4402 - val_loss: 0.6869 - val_acc: 0.5842 - val_f1: 0.3693
Epoch 3/50
44/44 [==============================] - 2s 55ms/step - loss: 0.6817 - acc: 0.5531 - f1: 0.4662 - val_loss: 0.6886 - val_acc: 0.5842 - val_f1: 0.3693
Epoch 4/50
44/44 [==============================] - 2s 54ms/step - loss: 0.6786 - acc: 0.5748 - f1: 0.4896 - val_loss: 0.6983 - val_acc: 0.5644 - val_f1: 0.3634
Epoch 5/50
44/44 [==============================] - 2s 55ms/step - loss: 0.6625 - acc: 0.5915 - f1: 0.5245 - val_loss: 0.7053 - val_acc: 0.4851 - val_f1: 0.4063
Epoch 6/50
44/44 [==============================] - 3s 58ms/step - loss: 0.6584 - acc: 0.6038 - f1: 0.5342 - val_loss: 

In [11]:
result_dji_full = model.predict(test_generator)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"DJI Accuracy: {accuracy_score(result_dji_full, y_test[win_length:])}")
print(f"DJI F1: {f1_score(result_dji_full, y_test[win_length:], average='macro')}")

DJI Accuracy: 0.6386554621848739
DJI F1: 0.42998774646318366


In [12]:
### NASDAQ
# Split
X = np.array(nasdaq_df_full.drop(columns=["MOVEMENT"]).copy())
y = np.array(nasdaq_df_full["MOVEMENT"].copy())

In [13]:
# Split train test, should not shuffle as the data is time series
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=False
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, shuffle=False
)

In [14]:
# Get input ready for lstm
win_length = 60
batch_size = 32
num_features = 82
train_generator = TimeseriesGenerator(np.array(X_train), np.array(y_train),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

test_generator = TimeseriesGenerator(np.array(X_test), np.array(y_test),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

valid_generator = TimeseriesGenerator(np.array(X_valid), np.array(y_valid),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

In [15]:
model = lstm(win_length, num_features)

optimizer = keras.optimizers.Adam()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                 patience=20,
                                                 mode="min")
model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit_generator(train_generator,
                   epochs=100,
                   validation_data=valid_generator,
                   shuffle=False, callbacks=[early_stopping])

Epoch 1/100
44/44 [==============================] - 3s 75ms/step - loss: 0.6998 - acc: 0.5127 - f1: 0.4150 - val_loss: 0.7324 - val_acc: 0.4257 - val_f1: 0.3183
Epoch 2/100
44/44 [==============================] - 3s 57ms/step - loss: 0.6995 - acc: 0.5278 - f1: 0.4041 - val_loss: 0.6816 - val_acc: 0.5743 - val_f1: 0.3650
Epoch 3/100
44/44 [==============================] - 2s 56ms/step - loss: 0.6842 - acc: 0.5459 - f1: 0.4458 - val_loss: 0.7011 - val_acc: 0.4752 - val_f1: 0.3658
Epoch 4/100
44/44 [==============================] - 3s 57ms/step - loss: 0.6738 - acc: 0.5705 - f1: 0.4713 - val_loss: 0.7576 - val_acc: 0.4455 - val_f1: 0.3617
Epoch 5/100
44/44 [==============================] - 3s 58ms/step - loss: 0.6663 - acc: 0.5958 - f1: 0.4911 - val_loss: 0.7936 - val_acc: 0.4158 - val_f1: 0.3375
Epoch 6/100
44/44 [==============================] - 3s 59ms/step - loss: 0.6565 - acc: 0.6023 - f1: 0.5200 - val_loss: 0.7321 - val_acc: 0.4356 - val_f1: 0.3640
Epoch 7/100
44/44 [=========

In [16]:
result_dji_full = model.predict(test_generator)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"NASDAQ Accuracy: {accuracy_score(result_dji_full, y_test[win_length:])}")
print(f"NASDAQ F1: {f1_score(result_dji_full, y_test[win_length:], average='macro')}")

NASDAQ Accuracy: 0.5294117647058824
NASDAQ F1: 0.48484848484848486


In [24]:
### NYSE
# Split
X = np.array(nyse_df_full.drop(columns=["MOVEMENT"]).copy())
y = np.array(nyse_df_full["MOVEMENT"].copy())

In [25]:
# Split train test, should not shuffle as the data is time series
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=False
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, shuffle=False
)

In [26]:
# Get input ready for lstm
win_length = 60
batch_size = 32
num_features = 82
train_generator = TimeseriesGenerator(np.array(X_train), np.array(y_train),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

test_generator = TimeseriesGenerator(np.array(X_test), np.array(y_test),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

valid_generator = TimeseriesGenerator(np.array(X_valid), np.array(y_valid),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

In [27]:
model = lstm(win_length, num_features)

optimizer = keras.optimizers.Adam()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                 patience=20,
                                                 mode="min")
model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit_generator(train_generator,
                   epochs=100,
                   validation_data=valid_generator,
                   shuffle=False, callbacks=[early_stopping])

Epoch 1/100
44/44 [==============================] - 3s 76ms/step - loss: 0.6979 - acc: 0.5155 - f1: 0.4361 - val_loss: 0.7236 - val_acc: 0.3960 - val_f1: 0.3129
Epoch 2/100
44/44 [==============================] - 3s 59ms/step - loss: 0.6966 - acc: 0.5365 - f1: 0.4181 - val_loss: 0.7304 - val_acc: 0.5644 - val_f1: 0.4057
Epoch 3/100
44/44 [==============================] - 3s 60ms/step - loss: 0.6807 - acc: 0.5611 - f1: 0.5090 - val_loss: 0.7263 - val_acc: 0.5347 - val_f1: 0.3530
Epoch 4/100
44/44 [==============================] - 2s 56ms/step - loss: 0.6782 - acc: 0.5748 - f1: 0.4830 - val_loss: 0.7372 - val_acc: 0.4950 - val_f1: 0.3816
Epoch 5/100
44/44 [==============================] - 2s 56ms/step - loss: 0.6716 - acc: 0.5777 - f1: 0.5284 - val_loss: 0.7849 - val_acc: 0.4752 - val_f1: 0.3837
Epoch 6/100
44/44 [==============================] - 2s 56ms/step - loss: 0.6680 - acc: 0.5936 - f1: 0.5259 - val_loss: 0.7603 - val_acc: 0.4554 - val_f1: 0.3655
Epoch 7/100
44/44 [=========

In [29]:
result_dji_full = model.predict(test_generator)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"NYSE Accuracy: {accuracy_score(result_dji_full, y_test[win_length:])}")
print(f"Nyse F1: {f1_score(result_dji_full, y_test[win_length:], average='macro')}")

NYSE Accuracy: 0.5294117647058824
Nyse F1: 0.4015804597701149


In [30]:
### russell
# Split
X = np.array(russell_df_full.drop(columns=["MOVEMENT"]).copy())
y = np.array(russell_df_full["MOVEMENT"].copy())

In [31]:
# Split train test, should not shuffle as the data is time series
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=False
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, shuffle=False
)

In [32]:
# Get input ready for lstm
win_length = 60
batch_size = 32
num_features = 82
train_generator = TimeseriesGenerator(np.array(X_train), np.array(y_train),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

test_generator = TimeseriesGenerator(np.array(X_test), np.array(y_test),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

valid_generator = TimeseriesGenerator(np.array(X_valid), np.array(y_valid),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

In [35]:
model = lstm(win_length, num_features)

optimizer = keras.optimizers.Adam()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                 patience=20,
                                                 mode="min")
model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit_generator(train_generator,
                   epochs=100,
                   validation_data=valid_generator,
                   shuffle=False, callbacks=[early_stopping])

Epoch 1/100
44/44 [==============================] - 3s 72ms/step - loss: 0.6992 - acc: 0.5018 - f1: 0.4315 - val_loss: 0.7200 - val_acc: 0.4257 - val_f1: 0.3286
Epoch 2/100
44/44 [==============================] - 2s 56ms/step - loss: 0.6938 - acc: 0.5322 - f1: 0.4562 - val_loss: 0.7104 - val_acc: 0.4752 - val_f1: 0.3503
Epoch 3/100
44/44 [==============================] - 3s 57ms/step - loss: 0.6844 - acc: 0.5430 - f1: 0.4757 - val_loss: 0.7186 - val_acc: 0.5149 - val_f1: 0.3868
Epoch 4/100
44/44 [==============================] - 3s 58ms/step - loss: 0.6762 - acc: 0.5719 - f1: 0.5176 - val_loss: 0.7485 - val_acc: 0.4752 - val_f1: 0.3742
Epoch 5/100
44/44 [==============================] - 3s 57ms/step - loss: 0.6702 - acc: 0.5842 - f1: 0.4982 - val_loss: 0.7322 - val_acc: 0.4455 - val_f1: 0.3365
Epoch 6/100
44/44 [==============================] - 3s 60ms/step - loss: 0.6690 - acc: 0.5922 - f1: 0.5322 - val_loss: 0.7374 - val_acc: 0.5347 - val_f1: 0.4452
Epoch 7/100
44/44 [=========

In [36]:
result_dji_full = model.predict(test_generator)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"RUSSELL Accuracy: {accuracy_score(result_dji_full, y_test[win_length:])}")
print(f"RUSSELL F1: {f1_score(result_dji_full, y_test[win_length:], average='macro')}")

RUSSELL Accuracy: 0.4957983193277311
RUSSELL F1: 0.3928571428571429


In [37]:
### sp
# Split
X = np.array(sp_df_full.drop(columns=["MOVEMENT"]).copy())
y = np.array(sp_df_full["MOVEMENT"].copy())

In [38]:
# Split train test, should not shuffle as the data is time series
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=False
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, shuffle=False
)

In [39]:
# Get input ready for lstm
win_length = 60
batch_size = 32
num_features = 82
train_generator = TimeseriesGenerator(np.array(X_train), np.array(y_train),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

test_generator = TimeseriesGenerator(np.array(X_test), np.array(y_test),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

valid_generator = TimeseriesGenerator(np.array(X_valid), np.array(y_valid),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

In [40]:
model = lstm(win_length, num_features)

optimizer = keras.optimizers.Adam()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                 patience=20,
                                                 mode="min")
model.compile(optimizer=optimizer, loss="binary_crossentropy", 
                   metrics=["acc", f1])
model.fit_generator(train_generator,
                   epochs=100,
                   validation_data=valid_generator,
                   shuffle=False, callbacks=[early_stopping])

Epoch 1/100
44/44 [==============================] - 4s 80ms/step - loss: 0.7006 - acc: 0.5083 - f1: 0.4192 - val_loss: 0.7391 - val_acc: 0.4554 - val_f1: 0.3417
Epoch 2/100
44/44 [==============================] - 3s 57ms/step - loss: 0.6929 - acc: 0.5416 - f1: 0.4342 - val_loss: 0.6947 - val_acc: 0.5149 - val_f1: 0.3789
Epoch 3/100
44/44 [==============================] - 3s 57ms/step - loss: 0.6767 - acc: 0.5777 - f1: 0.4848 - val_loss: 0.7045 - val_acc: 0.5248 - val_f1: 0.3895
Epoch 4/100
44/44 [==============================] - 2s 56ms/step - loss: 0.6690 - acc: 0.5799 - f1: 0.4958 - val_loss: 0.7087 - val_acc: 0.5149 - val_f1: 0.3976
Epoch 5/100
44/44 [==============================] - 3s 58ms/step - loss: 0.6624 - acc: 0.5936 - f1: 0.5347 - val_loss: 0.7185 - val_acc: 0.4752 - val_f1: 0.3895
Epoch 6/100
44/44 [==============================] - 3s 58ms/step - loss: 0.6408 - acc: 0.6327 - f1: 0.5881 - val_loss: 0.7495 - val_acc: 0.4950 - val_f1: 0.4319
Epoch 7/100
44/44 [=========

In [41]:
result_dji_full = model.predict(test_generator)
result_dji_full = (result_dji_full > 0.5).astype(int)
print(f"SP Accuracy: {accuracy_score(result_dji_full, y_test[win_length:])}")
print(f"SP F1: {f1_score(result_dji_full, y_test[win_length:], average='macro')}")

SP Accuracy: 0.5630252100840336
SP F1: 0.39285714285714285
